In [1]:
# klein beginnen, dan uitbouwen, 
# eerst nasdaq, dan twelve, dan nasdaq 100, etc etc
# eerst RSI, dan RSI en ... dan mischine boolean on or offs
# gridsearch dan bayesian, en ik geloof stiekem dat GP's mischien makkelijker zijn hahahhaha

In [2]:
TICKERS = [
    # Nasdaq Top 20
    "NVDA","MSFT","AAPL","GOOGL","AMZN","META","TSLA","AVGO","TSM","INTC",
    "ADBE","CMCSA","CSCO","PEP","TXN","PYPL","QCOM","AMGN","GILD","CHTR",

    # Europe Top 10
    "SAP","ASML","MC.PA","RMS.PA","OR.PA","AZN","VGK",

    # Asia Top 5
    "XOM","TSM","TCEHY","FXI","EWY",

    # Crypto
    "BTC-USD","ETH-USD"
]


In [3]:
API_KEY = "1VWXIAX2LM6F6WV0"
START_YEAR = 2010
TICKERS = ["AAPL","MSFT","NVDA","SPY","QQQ","VGK","FXI","EWY","XOM","TSM","BTC-USD","ETH-USD"]


In [6]:
import pandas as pd
import requests
import time
import json # Ensure json is imported for better error handling/parsing

# Updated fetch_alpha_vantage_data_robust function
def fetch_alpha_vantage_data_robust(ticker: str, api_key: str, output_size: str = "full") -> pd.DataFrame | None:
    """
    Fetch daily OHLCV data from Alpha Vantage for a single ticker (adjusted for stocks).
    Handles both TIME_SERIES_DAILY_ADJUSTED for stocks and DIGITAL_CURRENCY_DAILY for crypto.
    """
    base_url = "https://www.alphavantage.co/query"
    df = None

    def print_common_msgs(j: dict, ticker: str) -> bool:
        """Prints common Alpha Vantage messages (Note, Error Message, etc.) and indicates if it's a failure."""
        is_failure = False
        for k in ("Note", "Error Message", "Information", "Message"):
            if k in j:
                print(f"{ticker}: {k}: {j[k]}")
                if k in ("Error Message", "Information", "Message") or "frequency" in j[k].lower():
                    is_failure = True
        return is_failure

    # Crypto data
    if ticker.endswith("-USD"):
        params = {
            "function": "DIGITAL_CURRENCY_DAILY",
            "symbol": ticker.split("-")[0],
            "market": "USD",
            "apikey": api_key,
        }
        try:
            r = requests.get(base_url, params=params, timeout=20)
            r.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
            j = r.json()
        except requests.exceptions.RequestException as e:
            print(f"Request failed for {ticker}: {e}")
            return None
        except json.JSONDecodeError:
            print(f"{ticker}: Response not JSON: {r.text[:200]}")
            return None

        if print_common_msgs(j, ticker):
            return None
        
        key = "Time Series (Digital Currency Daily)"
        if key not in j:
            print(f"{ticker}: missing '{key}' in response for crypto")
            return None
        
        ts = j[key]
        df = pd.DataFrame.from_dict(ts, orient="index").astype(float)
        df.index = pd.to_datetime(df.index)
        
        # *** FIX HERE: Crypto close column is '4. close' based on your diagnostic ***
        close_col = "4. close"
        
        if close_col not in df.columns:
            print(f"{ticker}: close column not found in crypto payload. Available: {df.columns.tolist()}")
            return None

        df = df.rename(columns={close_col: "Close"})
        df = df[["Close"]].sort_index()
        df["Ticker"] = ticker
        return df

    # Stock data (adjusted for splits)
    params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": ticker,
        "outputsize": output_size,
        "apikey": api_key,
    }
    try:
        r = requests.get(base_url, params=params, timeout=20)
        r.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        j = r.json()
    except requests.exceptions.RequestException as e:
        print(f"Request failed for {ticker}: {e}")
        return None
    except json.JSONDecodeError:
        print(f"{ticker}: Response not JSON: {r.text[:200]}")
        return None

    if print_common_msgs(j, ticker):
        return None
    
    # Accept either "Time Series (Daily Adjusted)" or "Time Series (Daily)" keys
    key = None
    if "Time Series (Daily Adjusted)" in j:
        key = "Time Series (Daily Adjusted)"
    elif "Time Series (Daily)" in j:
        key = "Time Series (Daily)"
    
    if key is None:
        print(f"{ticker}: no daily time series key present in response")
        return None
    
    ts = j[key]
    df = pd.DataFrame.from_dict(ts, orient="index").astype(float)
    df.index = pd.to_datetime(df.index)
    
    # Prefer adjusted close when available, else fallback to raw close
    close_col = None
    if "5. adjusted close" in df.columns:
        close_col = "5. adjusted close"
    elif "4. close" in df.columns:
        close_col = "4. close"
    
    if close_col is None:
        print(f"{ticker}: no close column found (expected '5. adjusted close' or '4. close'). Available: {df.columns.tolist()}")
        return None
    
    df = df.rename(columns={close_col: "Close"})
    df = df[["Close"]].sort_index()
    df["Ticker"] = ticker
    return df

# Quick smoke test to verify API key and connectivity
print("\n--- Alpha Vantage Robust Fetcher Smoke Test ---")
# Ensure API_KEY is defined in your notebook, e.g., API_KEY = "YOUR_API_KEY_HERE"
if 'API_KEY' not in globals():
    print("⚠️ API_KEY not defined. Please define API_KEY = \"YOUR_ALPHA_VANTAGE_API_KEY\" in a previous cell.")
else:
    test_symbols = ["MSFT", "BTC-USD"]
    for sym in test_symbols:
        print(f"\nTesting {sym}...")
        out = fetch_alpha_vantage_data_robust(sym, API_KEY, output_size="compact") 
        if out is None:
            print(f"Result: {sym}: fetch failed.")
        else:
            print(f"Result: {sym}: rows={len(out)}, start={out.index.min().date()}, end={out.index.max().date()}")
        time.sleep(12) # Be nice to Alpha Vantage API (5 calls/min limit)
print("\n--- Smoke Test Complete ---")


--- Alpha Vantage Robust Fetcher Smoke Test ---

Testing MSFT...
Result: MSFT: rows=100, start=2025-03-27, end=2025-08-19

Testing BTC-USD...
Result: BTC-USD: rows=350, start=2024-09-05, end=2025-08-20

--- Smoke Test Complete ---


In [ ]:
alpha_vantage_diagnostic(API_KEY)


[STOCK MSFT] GET https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=MSFT&outputsize=compact&apikey=1VWXIAX2LM6F6WV0
Status: 200
Top-level keys: ['Meta Data', 'Time Series (Daily)']

[CRYPTO BTC-USD] GET https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=USD&apikey=1VWXIAX2LM6F6WV0
Status: 200
Top-level keys: ['Meta Data', 'Time Series (Digital Currency Daily)']
Data points: 350; first dates: ['2025-08-20', '2025-08-19', '2025-08-18']


In [ ]:
import pandas as pd
import requests
import time
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# -----------------------------
# Portfolio Tickers
# -----------------------------
TICKERS = [
    # Nasdaq Top 20
    "NVDA","MSFT","AAPL","GOOGL","AMZN","META","TSLA","AVGO","TSM","INTC",
    "ADBE","CMCSA","CSCO","PEP","TXN","PYPL","QCOM","AMGN","GILD","CHTR",

    # Europe Top 10
    "SAP","ASML","MC.PA","RMS.PA","OR.PA","AZN","VGK",

    # Asia Top 5
    "XOM","TSM","TCEHY","FXI","EWY",

    # Crypto
    "BTC-USD","ETH-USD"
]

# -----------------------------
# Fetching Function
# -----------------------------
def fetch_alpha_vantage_data(ticker, api_key, output_size='full'):
    base_url = "https://www.alphavantage.co/query"

    if ticker.endswith("-USD"):  # Crypto
        params = {
            "function": "DIGITAL_CURRENCY_DAILY",
            "symbol": ticker.split("-")[0],
            "market": "USD",
            "apikey": api_key
        }
    else:  # Stocks
        params = {
            'function': 'TIME_SERIES_DAILY_ADJUSTED',
            'symbol': ticker,
            'outputsize': output_size,
            'apikey': api_key,
            'datatype': 'json'
        }

    response = requests.get(base_url, params=params)
    data = response.json()

    # Stock data
    if "Time Series (Daily Adjusted)" in data:
        ts = data["Time Series (Daily Adjusted)"]
        df = pd.DataFrame.from_dict(ts, orient='index')
        df.index = pd.to_datetime(df.index)
        df = df.astype(float)
        df = df.rename(columns={'5. adjusted close': 'Close'})
        df['Ticker'] = ticker
        return df[['Close', 'Ticker']].sort_index()

    # Crypto data
    elif "Time Series (Digital Currency Daily)" in data:
        ts = data["Time Series (Digital Currency Daily)"]
        df = pd.DataFrame.from_dict(ts, orient='index')
        df.index = pd.to_datetime(df.index)
        df = df.astype(float)
        df = df.rename(columns={'4a. close (USD)': 'Close'})
        df['Ticker'] = ticker
        return df[['Close', 'Ticker']].sort_index()

    else:
        print(f"⚠️ Failed to fetch {ticker}: {data.get('Note', 'Unknown error')}")
        return None

# -----------------------------
# Fetch Data
# -----------------------------
API_KEY = "1VWXIAX2LM6F6WV0"
all_data = []

for ticker in TICKERS:
    print(f"Fetching {ticker}...")
    df = fetch_alpha_vantage_data(ticker, API_KEY)
    if df is not None:
        all_data.append(df)
    time.sleep(12)  # Alpha Vantage rate limit

if not all_data:
    raise RuntimeError("No data fetched.")

data = pd.concat(all_data)

# -----------------------------
# Plot Adjusted Close Prices
# -----------------------------
tickers = data['Ticker'].unique()
fig, axes = plt.subplots(len(tickers), 1, figsize=(14, 2*len(tickers)))
fig.suptitle('Adjusted Close Prices', fontsize=16, y=0.98)

if len(tickers) == 1:
    axes = [axes]

for i, ticker in enumerate(tickers):
    ax = axes[i]
    ticker_data = data[data['Ticker'] == ticker]

    if not ticker_data.index.is_unique:
        ticker_data = ticker_data.groupby(ticker_data.index).mean(numeric_only=True)

    ax.plot(ticker_data.index, ticker_data['Close'], label=ticker, linewidth=1.5)
    ax.set_title(ticker)
    ax.set_ylabel('Price')
    ax.grid(True, alpha=0.3)
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax.legend(loc='upper left')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


Fetching NVDA...
⚠️ Failed to fetch NVDA: Unknown error
Fetching MSFT...
⚠️ Failed to fetch MSFT: Unknown error
Fetching AAPL...
⚠️ Failed to fetch AAPL: Unknown error


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import requests
import time
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# -----------------------------
# Portfolio Tickers
# -----------------------------
TICKERS = [
    # Nasdaq Top 20
    "NVDA","MSFT","AAPL","GOOGL","AMZN","META","TSLA","AVGO","TSM","INTC",
    "ADBE","CMCSA","CSCO","PEP","TXN","PYPL","QCOM","AMGN","GILD","CHTR",

    # Europe Top 10
    "SAP","ASML","MC.PA","RMS.PA","OR.PA","AZN","VGK",

    # Asia Top 5
    "XOM","TSM","TCEHY","FXI","EWY",

    # Crypto
    "BTC-USD","ETH-USD"
]

# -----------------------------
# Fetching Function
# -----------------------------
def fetch_alpha_vantage_data(ticker, api_key, output_size='full'):
    base_url = "https://www.alphavantage.co/query"

    if ticker.endswith("-USD"):  # Crypto
        params = {
            "function": "DIGITAL_CURRENCY_DAILY",
            "symbol": ticker.split("-")[0],
            "market": "USD",
            "apikey": api_key
        }
    else:  # Stocks
        params = {
            'function': 'TIME_SERIES_DAILY_ADJUSTED',
            'symbol': ticker,
            'outputsize': output_size,
            'apikey': api_key,
            'datatype': 'json'
        }

    response = requests.get(base_url, params=params)
    data = response.json()

    # Stock data
    if "Time Series (Daily Adjusted)" in data:
        ts = data["Time Series (Daily Adjusted)"]
        df = pd.DataFrame.from_dict(ts, orient='index')
        df.index = pd.to_datetime(df.index)
        df = df.astype(float)
        df = df.rename(columns={'5. adjusted close': 'Close'})
        df['Ticker'] = ticker
        return df[['Close', 'Ticker']].sort_index()

    # Crypto data
    elif "Time Series (Digital Currency Daily)" in data:
        ts = data["Time Series (Digital Currency Daily)"]
        df = pd.DataFrame.from_dict(ts, orient='index')
        df.index = pd.to_datetime(df.index)
        df = df.astype(float)
        df = df.rename(columns={'4a. close (USD)': 'Close'})
        df['Ticker'] = ticker
        return df[['Close', 'Ticker']].sort_index()

    else:
        print(f"⚠️ Failed to fetch {ticker}: {data.get('Note', 'Unknown error')}")
        return None

# -----------------------------
# Fetch Data
# -----------------------------
API_KEY = "1VWXIAX2LM6F6WV0"
all_data = []

for ticker in TICKERS:
    print(f"Fetching {ticker}...")
    df = fetch_alpha_vantage_data(ticker, API_KEY)
    if df is not None:
        all_data.append(df)
    time.sleep(12)  # Alpha Vantage rate limit

if not all_data:
    raise RuntimeError("No data fetched.")

data = pd.concat(all_data)

# -----------------------------
# Plot Adjusted Close Prices
# -----------------------------
tickers = data['Ticker'].unique()
fig, axes = plt.subplots(len(tickers), 1, figsize=(14, 2*len(tickers)))
fig.suptitle('Adjusted Close Prices', fontsize=16, y=0.98)

if len(tickers) == 1:
    axes = [axes]

for i, ticker in enumerate(tickers):
    ax = axes[i]
    ticker_data = data[data['Ticker'] == ticker]

    if not ticker_data.index.is_unique:
        ticker_data = ticker_data.groupby(ticker_data.index).mean(numeric_only=True)

    ax.plot(ticker_data.index, ticker_data['Close'], label=ticker, linewidth=1.5)
    ax.set_title(ticker)
    ax.set_ylabel('Price')
    ax.grid(True, alpha=0.3)
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax.legend(loc='upper left')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


Fetching NVDA...
⚠️ Failed to fetch NVDA: Unknown error
Fetching MSFT...
⚠️ Failed to fetch MSFT: Unknown error
Fetching AAPL...
⚠️ Failed to fetch AAPL: Unknown error


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import requests
import time
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# -----------------------------
# Portfolio Tickers
# -----------------------------
TICKERS = [
    # Nasdaq Top 20
    "NVDA","MSFT","AAPL","GOOGL","AMZN","META","TSLA","AVGO","TSM","INTC",
    "ADBE","CMCSA","CSCO","PEP","TXN","PYPL","QCOM","AMGN","GILD","CHTR",

    # Europe Top 10
    "SAP","ASML","MC.PA","RMS.PA","OR.PA","AZN","VGK",

    # Asia Top 5
    "XOM","TSM","TCEHY","FXI","EWY",

    # Crypto
    "BTC-USD","ETH-USD"
]

# -----------------------------
# Fetching Function
# -----------------------------
def fetch_alpha_vantage_data(ticker, api_key, output_size='full'):
    base_url = "https://www.alphavantage.co/query"

    if ticker.endswith("-USD"):  # Crypto
        params = {
            "function": "DIGITAL_CURRENCY_DAILY",
            "symbol": ticker.split("-")[0],
            "market": "USD",
            "apikey": api_key
        }
    else:  # Stocks
        params = {
            'function': 'TIME_SERIES_DAILY_ADJUSTED',
            'symbol': ticker,
            'outputsize': output_size,
            'apikey': api_key,
            'datatype': 'json'
        }

    response = requests.get(base_url, params=params)
    data = response.json()

    # Stock data
    if "Time Series (Daily Adjusted)" in data:
        ts = data["Time Series (Daily Adjusted)"]
        df = pd.DataFrame.from_dict(ts, orient='index')
        df.index = pd.to_datetime(df.index)
        df = df.astype(float)
        df = df.rename(columns={'5. adjusted close': 'Close'})
        df['Ticker'] = ticker
        return df[['Close', 'Ticker']].sort_index()

    # Crypto data
    elif "Time Series (Digital Currency Daily)" in data:
        ts = data["Time Series (Digital Currency Daily)"]
        df = pd.DataFrame.from_dict(ts, orient='index')
        df.index = pd.to_datetime(df.index)
        df = df.astype(float)
        df = df.rename(columns={'4a. close (USD)': 'Close'})
        df['Ticker'] = ticker
        return df[['Close', 'Ticker']].sort_index()

    else:
        print(f"⚠️ Failed to fetch {ticker}: {data.get('Note', 'Unknown error')}")
        return None

# -----------------------------
# Fetch Data
# -----------------------------
API_KEY = "1VWXIAX2LM6F6WV0"
all_data = []

for ticker in TICKERS:
    print(f"Fetching {ticker}...")
    df = fetch_alpha_vantage_data(ticker, API_KEY)
    if df is not None:
        all_data.append(df)
    time.sleep(12)  # Alpha Vantage rate limit

if not all_data:
    raise RuntimeError("No data fetched.")

data = pd.concat(all_data)

# -----------------------------
# Plot Adjusted Close Prices
# -----------------------------
tickers = data['Ticker'].unique()
fig, axes = plt.subplots(len(tickers), 1, figsize=(14, 2*len(tickers)))
fig.suptitle('Adjusted Close Prices', fontsize=16, y=0.98)

if len(tickers) == 1:
    axes = [axes]

for i, ticker in enumerate(tickers):
    ax = axes[i]
    ticker_data = data[data['Ticker'] == ticker]

    if not ticker_data.index.is_unique:
        ticker_data = ticker_data.groupby(ticker_data.index).mean(numeric_only=True)

    ax.plot(ticker_data.index, ticker_data['Close'], label=ticker, linewidth=1.5)
    ax.set_title(ticker)
    ax.set_ylabel('Price')
    ax.grid(True, alpha=0.3)
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax.legend(loc='upper left')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


Fetching NVDA...
⚠️ Failed to fetch NVDA: Unknown error
Fetching MSFT...
⚠️ Failed to fetch MSFT: Unknown error
Fetching AAPL...
⚠️ Failed to fetch AAPL: Unknown error


KeyboardInterrupt: 

In [ ]:
# Get unique asset classes
asset_classes = data['AssetClass'].unique()

# Create subplots
fig, axes = plt.subplots(len(asset_classes), 1, figsize=(14, 3*len(asset_classes)))
fig.suptitle('Asset Class Performance', fontsize=16, y=0.98)

for i, asset in enumerate(asset_classes):
    ax = axes[i]
    asset_data = data[data['AssetClass'] == asset]
    
    # Plot closing price
    ax.plot(asset_data.index.get_level_values('Date'), 
            asset_data['Close'], 
            label=f'{asset} Close', 
            linewidth=1.5)
    
    # Formatting
    ax.set_title(asset)
    ax.set_ylabel('Price')
    ax.grid(True, alpha=0.3)
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    
    # Add legend
    ax.legend(loc='upper left')

plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust for suptitle
plt.show()

KeyError: 'AssetClass'

In [ ]:
import pandas as pd
import numpy as np

class Indicator:
    def __init__(self, data: pd.DataFrame):
        """
        Comprehensive technical indicator suite
        :param data: DataFrame with OHLCV columns and datetime index
        """
        self.data = data
    
    def rsi(self, period: int = 14) -> pd.Series:
        """Calculate the Relative Strength Index (RSI) with Wilder's smoothing."""
        delta = self.data['Close'].diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        
        avg_gain = gain.ewm(alpha=1/period, adjust=False).mean()
        avg_loss = loss.ewm(alpha=1/period, adjust=False).mean()
        
        rs = avg_gain / avg_loss
        rs = rs.replace([np.inf, -np.inf], np.nan).fillna(0)
        return 100 - (100 / (1 + rs))
    
    def adx(self, period: int = 14) -> pd.Series:
        """Calculate the Average Directional Index (ADX) for trend strength."""
        high, low, close = self.data['High'], self.data['Low'], self.data['Close']
        
        # Calculate directional movements
        up_move = high.diff()
        down_move = low.diff().abs()
        plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
        minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)
        
        # Calculate true range
        tr = pd.concat([
            high - low,
            (high - close.shift(1)).abs(),
            (low - close.shift(1)).abs()
        ], axis=1).max(axis=1)
        
        # Smooth values
        alpha = 1 / period
        atr = tr.ewm(alpha=alpha, adjust=False).mean()
        plus_di = 100 * plus_dm.ewm(alpha=alpha, adjust=False).mean() / atr
        minus_di = 100 * minus_dm.ewm(alpha=alpha, adjust=False).mean() / atr
        
        # Calculate DX and ADX
        dx = 100 * np.abs(plus_di - minus_di) / (plus_di + minus_di)
        return dx.ewm(alpha=alpha, adjust=False).mean()
    
    def kama(self, fast_period: int = 7, slow_period: int = 19, 
             er_period: int = 8, norm_period: int = 50, 
             normalize: bool = True) -> pd.Series:
        """
        Calculate Kaufman Adaptive Moving Average (KAMA) with normalization.
        :param fast_period: Fast EMA period (default 7)
        :param slow_period: Slow EMA period (default 19)
        :param er_period: Efficiency ratio period (default 8)
        :param norm_period: Normalization lookback (default 50)
        :param normalize: Whether to normalize the oscillator (default True)
        :return: Normalized KAMA oscillator series
        """
        close = self.data['Close']
        
        # Calculate efficiency ratio
        change = (close - close.shift(er_period)).abs()
        volatility = close.diff().abs().rolling(window=er_period).sum()
        er = change / volatility
        er = er.replace([np.inf, -np.inf], np.nan).fillna(0)
        
        # Calculate smoothing constant
        sc = er * (2/(fast_period + 1) - 2/(slow_period + 1)) + 2/(slow_period + 1)
        
        # Calculate KAMA
        kama = pd.Series(np.nan, index=close.index)
        kama.iloc[0] = close.iloc[0]
        
        for i in range(1, len(close)):
            kama.iloc[i] = kama.iloc[i-1] + sc.iloc[i] * (close.iloc[i] - kama.iloc[i-1])
        
        if not normalize:
            return kama
        
        # Normalize the oscillator
        lowest = kama.rolling(window=norm_period).min()
        highest = kama.rolling(window=norm_period).max()
        range_val = highest - lowest
        normalized = (kama - lowest) / range_val - 0.5
        normalized = normalized.replace([np.inf, -np.inf], np.nan).fillna(0)
        
        return normalized
    
    def atr(self, period: int = 14) -> pd.Series:
        """
        Calculate Average True Range (ATR) for volatility measurement.
        :param period: Lookback period for ATR calculation
        :return: Series of ATR values
        """
        high, low, close = self.data['High'], self.data['Low'], self.data['Close']
        
        # Calculate true range
        tr1 = high - low
        tr2 = (high - close.shift(1)).abs()
        tr3 = (low - close.shift(1)).abs()
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        
        # Calculate ATR with Wilder's smoothing
        return tr.ewm(alpha=1/period, adjust=False).mean()
    
    def mfi(self, period: int = 14) -> pd.Series:
        """
        Calculate Money Flow Index (MFI) for volume-based momentum.
        :param period: Lookback period for MFI calculation
        :return: Series of MFI values (0-100)
        """
        high, low, close, volume = self.data['High'], self.data['Low'], self.data['Close'], self.data['Volume']
        
        # Calculate typical price
        typical_price = (high + low + close) / 3
        
        # Calculate raw money flow
        money_flow = typical_price * volume
        
        # Determine positive/negative money flow
        price_diff = typical_price.diff()
        positive_flow = np.where(price_diff > 0, money_flow, 0)
        negative_flow = np.where(price_diff < 0, money_flow, 0)
        
        # Calculate money flow ratio
        pos_flow_sum = pd.Series(positive_flow).rolling(window=period).sum()
        neg_flow_sum = pd.Series(negative_flow).rolling(window=period).sum()
        money_ratio = pos_flow_sum / neg_flow_sum
        
        # Calculate MFI
        mfi = 100 - (100 / (1 + money_ratio))
        return mfi.replace([np.inf, -np.inf], np.nan).fillna(50)  # 50 is neutral
    
    def entropy(self, window: int = 10, bins: int = 5) -> pd.Series:
        """Calculate Shannon's Entropy of returns distribution."""
        returns = self.data['Close'].pct_change().dropna()
        entropy_vals = pd.Series(np.nan, index=self.data.index)
        
        for i in range(window, len(returns)):
            window_returns = returns.iloc[i-window:i]
            hist, _ = np.histogram(window_returns, bins=bins)
            prob = hist / hist.sum()
            entropy_vals.iloc[i] = entropy(prob)
        
        return entropy_vals

In [ ]:
# buying and selling logic of indicators

In [ ]:
# individueel plotten met buy strats gewoon voor visual en check


In [ ]:
# 2018 tot 2022 modelleren wellicht en out of sample er voor of er na chechken,

# rollen sharpe en marco lopez sharpe checks

In [ ]:
# Robust Alpha Vantage fetcher (handles both Daily and Daily Adjusted keys)
import pandas as pd
import requests
import time

def fetch_alpha_vantage_data_robust(ticker: str, api_key: str, output_size: str = "full") -> pd.DataFrame | None:
    base_url = "https://www.alphavantage.co/query"

    def print_common_msgs(j: dict, ticker: str) -> bool:
        for k in ("Note", "Error Message", "Information", "Message"):
            if k in j:
                print(f"{ticker}: {k}: {j[k]}")
                # Return True if this likely indicates a failure
                if k in ("Error Message", "Information", "Message") or "frequency" in j[k].lower():
                    return True
        return False

    # Crypto
    if ticker.endswith("-USD"):
        params = {
            "function": "DIGITAL_CURRENCY_DAILY",
            "symbol": ticker.split("-")[0],
            "market": "USD",
            "apikey": api_key,
        }
        r = requests.get(base_url, params=params, timeout=20)
        j = r.json()
        if print_common_msgs(j, ticker):
            return None
        key = "Time Series (Digital Currency Daily)"
        if key not in j:
            print(f"{ticker}: missing '{key}' in response")
            return None
        ts = j[key]
        df = pd.DataFrame.from_dict(ts, orient="index").astype(float)
        df.index = pd.to_datetime(df.index)
        close_col = "4a. close (USD)" if "4a. close (USD)" in df.columns else "4b. close (USD)"
        if close_col not in df.columns:
            print(f"{ticker}: close column not found in crypto payload")
            return None
        df = df.rename(columns={close_col: "Close"})
        df = df[["Close"]].sort_index()
        df["Ticker"] = ticker
        return df

    # Stocks
    params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": ticker,
        "outputsize": output_size,
        "apikey": api_key,
    }
    r = requests.get(base_url, params=params, timeout=20)
    j = r.json()
    if print_common_msgs(j, ticker):
        # If rate-limited, caller can sleep and retry
        pass
    # Accept either key name
    key = "Time Series (Daily Adjusted)" if "Time Series (Daily Adjusted)" in j else "Time Series (Daily)" if "Time Series (Daily)" in j else None
    if key is None:
        print(f"{ticker}: no daily time series key present")
        return None
    ts = j[key]
    df = pd.DataFrame.from_dict(ts, orient="index").astype(float)
    df.index = pd.to_datetime(df.index)
    # Prefer adjusted close when available, else fallback to raw close
    close_col = "5. adjusted close" if "5. adjusted close" in df.columns else ("4. close" if "4. close" in df.columns else None)
    if close_col is None:
        print(f"{ticker}: no close column found (expected '5. adjusted close' or '4. close')")
        return None
    df = df.rename(columns={close_col: "Close"})
    df = df[["Close"]].sort_index()
    df["Ticker"] = ticker
    return df

# Quick smoke test
print("\nRobust fetcher smoke test (MSFT, BTC-USD):")
for sym in ["MSFT", "BTC-USD"]:
    out = fetch_alpha_vantage_data_robust(sym, API_KEY, output_size="compact") if 'API_KEY' in globals() else None
    if out is None:
        print(f"{sym}: fetch failed")
    else:
        print(f"{sym}: rows={len(out)}, start={out.index.min().date()}, end={out.index.max().date()}")
        # Be nice to API
        time.sleep(12)


In [ ]:
# Alpha Vantage API diagnostics
import requests, json

def alpha_vantage_diagnostic(api_key: str, stock_symbol: str = "MSFT", crypto_symbol: str = "BTC") -> None:
    """
    Make simple test calls to Alpha Vantage and print rich diagnostics so you can see
    whether the API is reachable, if you're rate-limited, or if the key is invalid.
    """
    base_url = "https://www.alphavantage.co/query"

    def do_call(params: dict, label: str) -> None:
        try:
            r = requests.get(base_url, params=params, timeout=20)
            print(f"\n[{label}] GET {r.url}")
            print(f"Status: {r.status_code}")
            try:
                j = r.json()
            except ValueError:
                print("Response not JSON. First 200 chars:")
                print(r.text[:200])
                return
            # Top-level keys present in the response
            print("Top-level keys:", list(j.keys()))
            # Common informational/error fields Alpha Vantage returns
            for k in ("Note", "Error Message", "Information", "Message"):
                if k in j:
                    print(f"{k}: {j[k]}")
            # If data is present, show a tiny sample
            for k in ("Time Series (Daily Adjusted)", "Time Series (Digital Currency Daily)"):
                if k in j:
                    ts = j[k]
                    keys = list(ts.keys())
                    print(f"Data points: {len(keys)}; first dates: {keys[:3]}")
        except Exception as e:
            print(f"Request failed: {e}")

    # Test a stock endpoint
    do_call(
        {
            "function": "TIME_SERIES_DAILY_ADJUSTED",
            "symbol": stock_symbol,
            "outputsize": "compact",
            "apikey": api_key,
        },
        f"STOCK {stock_symbol}",
    )

    # Test a crypto endpoint
    do_call(
        {
            "function": "DIGITAL_CURRENCY_DAILY",
            "symbol": crypto_symbol,
            "market": "USD",
            "apikey": api_key,
        },
        f"CRYPTO {crypto_symbol}-USD",
    )

print("Run alpha_vantage_diagnostic(API_KEY) to test connectivity and rate limits.")
# Example:
# alpha_vantage_diagnostic(API_KEY)
